In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%cd /home/ubuntu/projects/hyper-sloth

In [2]:
from fastcore.all import *
from speedy_utils.all import *
from llm_utils import *

#### Create a LLM model

In [3]:
import os

# os.environ["CUDA_VISIBLE_DEVICES"] = "01"  # Choose any GPU you want to use
from vllm.lora.request import LoRARequest
from vllm import LLM, SamplingParams
import torch

if "llm" in locals():
    del llm  #
    torch.cuda.empty_cache()
    
llm = LLM(
    # model="outputs/lora/Qwen2.5-1.5B-Instruct-LORA-MATH-merge",
    model="./outputs/loras/qwen1.5-openr1-1card/checkpoint-2500-merged",
    tensor_parallel_size=1,
    task="generate",
    enforce_eager=True,
    dtype=torch.bfloat16,
    max_model_len=16384,
    enable_lora=True,
    # quantization="bitsandbytes", load_format="bitsandbytes",gpu_memory_utilization=0.95
)

tokenizer = llm.get_tokenizer()

from datasets import load_dataset

# Load the GSM8K dataset
gsm8k = load_dataset("gsm8k", "main")
test = gsm8k["test"]


INFO 03-17 02:52:25 __init__.py:207] Automatically detected platform cuda.
WARNING 03-17 02:52:25 config.py:2448] Casting torch.float16 to torch.bfloat16.
WARNING 03-17 02:52:33 config.py:628] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 03-17 02:52:33 cuda.py:95] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
WARNING 03-17 02:52:33 config.py:685] Async output processing is not supported on the current platform type cuda.
WARNING 03-17 02:52:33 config.py:2217] bitsandbytes quantization is not tested with LoRA yet.
INFO 03-17 02:52:33 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='./outputs/loras/qwen1.5-openr1-1card/checkpoint-2500-merged', speculative_config=None, tokenizer='./outputs/loras/qwen1.5-openr1-1card/checkpoint-2500-merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, overr

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]


AssertionError: 

In [4]:
%debug

> /home/ubuntu/.conda/envs/vllm/lib/python3.10/site-packages/vllm/model_executor/layers/linear.py(1121)weight_loader()
   1119             loaded_weight = loaded_weight.reshape(1)
   1120 
-> 1121         assert param_data.shape == loaded_weight.shape
   1122         param_data.copy_(loaded_weight)
   1123 

*** AttributeError: 'torch.Size' object has no attribute 'shape'
torch.Size([6881280, 1])
tensor([[ 0.0151,  0.0259, -0.0023,  ..., -0.0058,  0.0075, -0.0126],
        [ 0.0112, -0.0059, -0.0138,  ..., -0.0002,  0.0516, -0.0066],
        [ 0.0073, -0.0156, -0.0197,  ...,  0.0091, -0.0239,  0.0269],
        ...,
        [-0.0070, -0.0469, -0.0205,  ...,  0.0039,  0.0601, -0.0059],
        [-0.0210, -0.0292,  0.0323,  ..., -0.0004, -0.0224,  0.0067],
        [ 0.0055,  0.0243,  0.0258,  ...,  0.0150,  0.0146, -0.0266]],
       dtype=torch.float32)
torch.Size([1536, 8960])
13762560
torch.Size([6881280, 1])


In [9]:
# Prepare prompts for GSM8K evaluation
all_questions = [item["question"] for item in test][:100]
standardized_prompts = [
    tokenizer.apply_chat_template(
        [
            {
                "role": "user",
                "content": f"{question}\nSolve step by step and put your final numerical answer inside \\boxed{{}}",
            }
        ],
        tokenize=False,
        add_generation_prompt=True,
    )
    for question in all_questions
]

# Set sampling parameters for deterministic generation
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.95,
    top_k=64,
    max_tokens=10000,
)
# Generate responses for all questions
outputs = llm.generate(
    standardized_prompts,
    sampling_params,
    lora_request=LoRARequest(
        "math", 1, "./outputs/loras/qwen1.5-openr1/checkpoint-732/"
    ),
)
all_outputs = [output.outputs[0].text for output in outputs]

Processed prompts: 100%|██████████| 100/100 [06:30<00:00,  3.91s/it, est. speed input: 26.43 toks/s, output: 645.11 toks/s]


In [20]:
all_questions[0]

"Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"

In [10]:
def get_final_output(response):
    try:
        return int(response.split("\\boxed{")[1].split("}")[0])
    except:
        return None

In [11]:
final_outputs = [get_final_output(response) for response in all_outputs]
accs = []
num_error = 0
for i, gt in enumerate(test):
    if i >= len(final_outputs):
        break
    pred = final_outputs[i]
    try:
        num = gt['answer'].split('####')[1]
        num = int(num)
        pred = int(pred)
        accs.append(num == pred)
    except:
        num_error += 1
        accs.append(0)
        pass

In [12]:
np.mean(accs), num_error/len(final_outputs)

(0.51, 0.28)

In [21]:
print(all_outputs[0])

<think>
Okay, let's see. Janet has ducks that lay 16 eggs per day. That's the first piece of information. Then, she eats three eggs for breakfast every morning. The next part says she bakes muffins for her friends every day with four eggs. After that, she sells the remainder at the farmers' market for $2 per egg. We need to find out how much she makes every day at the market.

Alright, so first, let me figure out the total number of eggs laid each day. Since the ducks lay 16 eggs, that's straightforward: 16 eggs. Then, Janet eats three of them for breakfast. So, how many eggs are left after breakfast? That would be 16 minus 3, which is 13 eggs. So, 13 eggs are left for baking muffins.

Then, she bakes four eggs for her friends. So, after baking muffins, how many eggs are left? That's 13 minus 4, which is 9 eggs. So, 9 eggs are left to sell at the market.

Now, the question is, how much does she make from selling these 9 eggs? Each egg is $2. So, 9 eggs times $2 per egg. Let me calculat